# **Predicción del campeón de copa américa USA 2024 🏆 ⚽ 🏃🏻**

####Paso 1: Importamos las librerías.


In [ ]:
import pandas as pd
import pickle
from scipy.stats import poisson

####Paso 2: Nos conectamos a Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


####Paso 3: Importamos el Dataset


In [ ]:
dict_table = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/DS - MUNDIAL FIFA QATAR 2022/Data limpia de Frank Andrade/dict_table','rb'))
df_historical_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS - MUNDIAL FIFA QATAR 2022/Data limpia de Frank Andrade/clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS - MUNDIAL FIFA QATAR 2022/Data limpia de Frank Andrade/clean_fifa_worldcup_fixture.csv')

* Revisamos dict_table:

In [ ]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


* Revisamos df_historical_data:

In [ ]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


* Revisamos df_fixture

In [ ]:
df_fixture

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


####Paso 3: Importamos el Dataset

####Paso 4. Calcular la fuerza de cada equipo

* Creamos un dataFrame que indicará la fuerza de cada equipo:

#### Paso 5. Dividir el df histórico en df_home and df_away

In [ ]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [ ]:
df_home

,HomeTeam,HomeGoals,AwayGoals
0,France,4,1
1,Uruguay,4,2
2,Uruguay,6,1
3,Argentina,6,1
4,Paraguay,1,0
...,...,...,...
895,Brazil,2,0
896,Serbia,1,2
897,Serbia,0,2
898,France,1,0


* Renombrar columnas:

In [ ]:
df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

In [ ]:
pd.concat([df_home, df_away], ignore_index=True)

,Team,GoalsScored,GoalsConceded
0,France,4,1
1,Uruguay,4,2
2,Uruguay,6,1
3,Argentina,6,1
4,Paraguay,1,0
...,...,...,...
1795,Costa Rica,0,2
1796,Switzerland,2,1
1797,Brazil,2,0
1798,Peru,0,1


In [ ]:
pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


####Paso 6: Ahora agrupamos por Team y le aplicamos el promedio (con esto calculamos el promedio de goles anotados y recibidos por cada selección)

* Con esto veremos la fortaleza y debilidades de cada equipo:

In [ ]:
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


#### Paso 6. Función predict_points

* Función que predice la cantidad de puntos para cada equipo:



In [ ]:
def predict_points(home, away):

  #Condicional para comprobar si home y away está dentro del df_team_strength
    if home in df_team_strength.index and away in df_team_strength.index:

        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0

        for x in range(0,11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)

    else:
        return (0, 0)

#### Paso 7. Testear función

In [ ]:
print(predict_points('Argentina', 'Mexico'))

(2.3129151525530505, 0.5378377125059863)


#### 8. Prediciendo el Mundial
* Empezando con la predicción simulando la fase de grupos

#### 8.1. Fase de Grupos

In [ ]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [ ]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [ ]:
df_fixture

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [ ]:
df_fixture_group_48

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [ ]:
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [ ]:
#Correr todos los partidos de la fase de grupo y actualizar las tablas de cada grupo
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values

    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]

    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']

        points_home, points_away = predict_points(home, away)

        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']] #solo mostramos estas dos columnas
    dict_table[group] = dict_table[group].round(0) #redondeamos a cero decimales

In [ ]:
#Mostrar tabla actualizada:
dict_table['Group A']

,Team,Pts
0,Netherlands,8.0
1,Senegal,4.0
2,Ecuador,4.0
3,Qatar (H),0.0


#### 8.2. Octavos

In [ ]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [ ]:
#Actualizar el ficture de octavos con el 1er puesto (group winner) y 2do puesto (runners up)
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']

    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

#Agregamos una nueva columna winner:
df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [ ]:
#Función para determinar el ganador de un partido:

def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows(): #iterrows(): itera cada fila del df
    # tomamos el nombre del equipo local y el nombre del equipo visitante:
        home, away = row['home'], row['away']
        #Llamamos al a función predict_points en viandole el nombre de cada equipo.
        points_home, points_away = predict_points(home, away)
        #Si los puntos del local son mayores al del visitante entonces gana el local:
        if points_home > points_away:
            winner = home
        #Si los puntos del local son menores al del visitante entonces gana el visitante:
        else:
            winner = away

        df_fixture_updated.loc[index, 'winner'] = winner

    return df_fixture_updated

In [ ]:
#Llamamos a la función obtener ganador (get_winner)
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


#### 8.3. Quarter Final

In [ ]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        #identificamos el ganador del partido:
        winner = df_fixture_round_1.loc[index, 'winner']
        #identificamos el número del partido:
        match = df_fixture_round_1.loc[index, 'score']
        #reemplazar la palabra "Winner + numero del partido" y reemplazarlo por la variable winner
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [ ]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [ ]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


#### 8.4. Semifinal

In [ ]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [ ]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


#### 8.5. Final

In [ ]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [ ]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
